## Fundamental Law of Memory Recall ( Parallel Implementation)

In [38]:
import numpy as np
import time
import numpy.random as npr
import matplotlib.pyplot as plt
import dask
from dask.distributed import Client, LocalCluster

In [23]:
def m_th_largest_index(nums , m):
    
    # Convert the list to a numpy array
    arr = np.array(nums)
    
    # Find the indices that would sort the array in descending order
    sorted_indices = np.argsort(arr)[::-1]
    
    # Return the index of the mth largest number
    return sorted_indices[m - 1]

In [28]:
def sequencer(sm):
    
    sequence_length = 3 *  len(sm)
    sequence = np.zeros(sequence_length , dtype = int)
    
    for i in range(sequence_length - 1):
        
        first_largest_index = m_th_largest_index(sm[sequence[i]] , 2)
        second_largest_index = m_th_largest_index(sm[sequence[i]] , 3)
        
        if first_largest_index != sequence[i-1]: sequence[i+1] = first_largest_index
        else: sequence[i+1] = second_largest_index
            
    return sequence

In [29]:
def process_iteration(i, f, N, L):

    mrs = np.array([[1 if npr.rand() < f else 0 for _ in range(N)] for _ in range(L)])
    sm = np.array([[np.dot(mrs[k], mrs[j]) for j in range(L)] for k in range(L)])
    return len(np.unique(sequencer(sm)))

In [30]:
def process_iteration_wrapper(args):
    return process_iteration(*args)

In [60]:
# Create a local cluster with multiple processes (or threads)

cluster = LocalCluster(n_workers=4, threads_per_worker=1)
client = Client(cluster)

C:\ProgramData\Anaconda3\lib\site-packages\distributed\node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 3729 instead
  warnings.warn(


Dask Dashboard URL: http://127.0.0.1:3729/status


In [64]:
cluster

2024-01-01 13:36:11,599 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:2298', name: 2, status: running, memory: 0, processing: 0>
2024-01-01 13:36:11,613 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:2238', name: 0, status: running, memory: 0, processing: 0>
2024-01-01 13:36:11,614 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:2304', name: 0, status: running, memory: 0, processing: 0>
2024-01-01 13:36:12,104 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:2244', name: 2, status: running, memory: 0, processing: 0>
2024-01-01 13:36:12,109 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:2304'.
2024-01-01 13:36:12,178 - distributed.scheduler - WA

In [61]:
f = 0.01
T = 20
N = 100000
L = 500


futures = client.map(process_iteration_wrapper, [(i, f, N, L) for i in range(T)])

In [62]:
start_time = time.time() 

results = client.gather(futures)

end_time = time.time()  

elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Elapsed time: 298.81 seconds


In [63]:
Rs = np.array(results)
print(np.mean(Rs), np.std(Rs))

35.85 19.2204968718293


In [59]:
client.close()
cluster.close()